In [2]:
%pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 11.4 MB/s eta 0:00:00


In [3]:
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

In [4]:
df=pd.read_csv('/content/train_data_attrition_scaling.csv')
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,-0.953774,1,-0.040465,2,-1.013473,2,1,1,1927,3,...,80,1,-0.527297,-0.613546,3,0.011597,0.772856,-0.676110,0.843327,0
1,0.241886,1,-0.878823,1,-0.154619,1,1,1,602,3,...,80,1,-0.399283,0.157319,3,0.175907,-0.334520,-0.676110,0.843327,0
2,0.024494,2,0.968031,1,-0.522699,2,3,1,460,4,...,80,2,-0.399283,-0.613546,2,-0.152713,-0.611364,-0.676110,-0.007719,0
3,-0.192899,2,0.844743,1,1.685783,4,3,1,75,3,...,80,1,-0.911340,0.157319,3,-0.809953,-0.611364,-0.034378,-0.575084,0
4,-0.084203,2,-0.723480,1,-0.768086,3,5,1,397,3,...,80,0,-0.143254,-0.613546,3,0.175907,-1.165051,1.569949,0.843327,0


In [8]:
X = df.drop(columns=['Attrition'])
y = df['Attrition']

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-2, 1e2)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
    gamma = 'scale'
    if kernel in ['rbf', 'poly', 'sigmoid']:
        gamma = trial.suggest_loguniform('gamma', 1e-3, 1)
    clf = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(C=C, kernel=kernel, gamma=gamma, probability=True, random_state=42))
    ])
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    y_proba = cross_val_predict(clf, X, y, cv=skf, method='predict_proba')[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)
    f1 = f1_score(y, y_pred)
    return f1
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("Best F1-score:", study.best_value)
print("Best parameters:", study.best_params)

[I 2025-06-13 00:48:11,975] A new study created in memory with name: no-name-b2165632-13c2-495a-8848-629b989d486e
<ipython-input-8-708623800>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-2, 1e2)
[I 2025-06-13 00:48:12,592] Trial 0 finished with value: 0.8246385920804525 and parameters: {'C': 0.06721538780749109, 'kernel': 'linear'}. Best is trial 0 with value: 0.8246385920804525.
<ipython-input-8-708623800>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-2, 1e2)
[I 2025-06-13 00:48:13,300] Trial 1 finished with value: 0.8215403882279274 and parameters: {'C': 0.013722779802275301, 'kernel': 'l

Best F1-score: 0.9405684754521964
Best parameters: {'C': 23.9362999026375, 'kernel': 'rbf', 'gamma': 0.1331531291014342}


In [9]:
best_params = study.best_params
kernel = best_params['kernel']
gamma = best_params.get('gamma', 'scale')

clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(C=best_params['C'], kernel=kernel, gamma=gamma, probability=True, random_state=42))
])
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_proba = cross_val_predict(clf, X, y, cv=skf, method='predict_proba')[:, 1]
y_pred = (y_proba >= 0.5).astype(int)
print("Precision:", precision_score(y, y_pred))
print("Recall:", recall_score(y, y_pred))
print("F1-score:", f1_score(y, y_pred))
print("ROC AUC:", roc_auc_score(y, y_proba))


Precision: 0.9561855670103093
Recall: 0.9428208386277002
F1-score: 0.9494561740243123
ROC AUC: 0.9882008947816245
